In [1]:
import json
import pandas as pd
import numpy as np

# Load player season history and create features

In [2]:
# Load all player season history into a single data frame
with open('data/during-season/bootstrap-static.json', 'r') as json_file:
    data = json.loads(json_file.read())
    
df_player_overview = pd.json_normalize(data, record_path='elements')

df_players_history = pd.DataFrame()
for index, player in df_player_overview.iterrows():
    player_id = str(player['id'])
    first_name = player['first_name']
    last_name = player['second_name']
    team_id = player['team']
    file_name = f"{player_id.rjust(3, '0')}_{first_name}_{last_name}".replace(' ', '_')

    with open(f'data/during-season/players/{file_name}.json', 'r') as json_file:
        player_data = json.loads(json_file.read())
        
    df_player_history = pd.json_normalize(player_data, record_path='history')
    df_player_history['name'] = f'{first_name} {last_name}'
    df_player_history['team_id'] = team_id
        
    df_players_history = df_players_history.append(df_player_history, ignore_index=True)

# Rename columns
df_players_history.rename(columns={
    'element': 'player_id',
    'opponent_team': 'opponent_team_id'
}, inplace=True)

# Set correct data types
df_players_history['round'] = df_players_history['round'].astype(int)
df_players_history['player_id'] = df_players_history['player_id'].astype(int)
df_players_history['opponent_team_id'] = df_players_history['opponent_team_id'].astype(int)
    
df_players_history.shape

(13358, 35)

In [3]:
df_players_history.sample(3).T

,10291,8247,2639
player_id,368,295,95
fixture,128,123,45
opponent_team_id,3,16,14
total_points,0,0,0
was_home,False,False,False
kickoff_time,2022-07-31T13:00:00Z,2022-07-30T13:00:00Z,2022-05-01T13:00:00Z
team_h_score,0,2,2
team_a_score,1,1,1
round,16,16,6
minutes,0,0,0


In [4]:
# Load teams data
df_teams = pd.json_normalize(data, record_path='teams')
df_teams.set_index('id', inplace=True)
df_teams

,code,draw,form,loss,name,played,points,position,short_name,strength,team_division,unavailable,win
id,,,,,,,,,,,,,
1,993,0,None,0,Malmö FF,0,0,0,MFF,None,None,False,0
2,1987,0,None,0,Hammarby,0,0,0,HAM,None,None,False,0
3,3056,0,None,0,IK Sirius,0,0,0,IKS,None,None,False,0
4,244,0,None,0,IFK Göteborg,0,0,0,GBG,None,None,False,0
5,375,0,None,0,IF Elfsborg,0,0,0,IFE,None,None,False,0
6,1959,0,None,0,BK Häcken,0,0,0,BKH,None,None,False,0
7,1406,0,None,0,Djurgården,0,0,0,DIF,None,None,False,0
8,2596,0,None,0,Örebro SK,0,0,0,ÖSK,None,None,True,0
9,483,0,None,0,IFK Norrköping,0,0,0,NOR,None,None,False,0


In [5]:
# Map team ids to team names for better readability
df_players_history['team'] = df_players_history['team_id'].map(df_teams['name'])
df_players_history['opponent_team'] = df_players_history['opponent_team_id'].map(df_teams['name'])

In [6]:
df_players_history.tail(5).T

,13353,13354,13355,13356,13357
player_id,526,527,528,529,530
fixture,218,217,218,221,222
opponent_team_id,17,6,17,4,9
total_points,0,0,0,0,0
was_home,True,False,True,False,True
kickoff_time,2022-10-24T17:00:00Z,2022-10-23T15:30:00Z,2022-10-24T17:00:00Z,2022-10-24T17:10:00Z,2022-10-24T17:00:00Z
team_h_score,4,2,4,1,1
team_a_score,0,1,0,0,1
round,28,28,28,28,28
minutes,0,0,0,0,0


In [7]:
df_players_history.nunique()

player_id                           530
fixture                             224
opponent_team_id                     16
total_points                         26
was_home                              2
kickoff_time                        134
team_h_score                          6
team_a_score                          7
round                                28
minutes                              89
goals_scored                          4
assists                               4
clean_sheets                          2
goals_conceded                        7
penalties_saved                       2
penalties_missed                      2
yellow_cards                          2
red_cards                             2
saves                                14
own_goals                             2
attacking_bonus                       3
defending_bonus                       3
winning_goals                         2
crosses                              10
key_passes                            9


In [8]:
df_players_history.to_csv('data/during-season/player_history.csv', index=False)

## Create data file for player future events

In [9]:
# Load all player season future events into a single csv
with open('data/during-season/bootstrap-static.json', 'r') as json_file:
    data = json.loads(json_file.read())
    
df_player_overview = pd.json_normalize(data, record_path='elements')

df_all_player_fixtures = pd.DataFrame()
for index, player in df_player_overview.iterrows():
    player_id = str(player['id'])
    first_name = player['first_name']
    last_name = player['second_name']
    team_id = player['team']
    file_name = f"{player_id.rjust(3, '0')}_{first_name}_{last_name}".replace(' ', '_')

    with open(f'data/during-season/players/{file_name}.json', 'r') as json_file:
        player_data = json.loads(json_file.read())
        
    df_player_fixtures = pd.json_normalize(player_data, record_path='fixtures')
    df_player_fixtures['player_id'] = player_id
    df_player_fixtures['name'] = f'{first_name} {last_name}'
        
    df_all_player_fixtures = df_all_player_fixtures.append(df_player_fixtures, ignore_index=True)

In [10]:
# Set home and away team
df_all_player_fixtures['team_id'] = np.where(df_all_player_fixtures['is_home'], df_all_player_fixtures['team_h'], df_all_player_fixtures['team_a'])
df_all_player_fixtures['opponent_team_id'] = np.where(df_all_player_fixtures['is_home'] == False, df_all_player_fixtures['team_h'], df_all_player_fixtures['team_a'])

# Map team ids to team names for better readability
df_all_player_fixtures['team'] = df_all_player_fixtures['team_id'].map(df_teams['name'])
df_all_player_fixtures['opponent_team'] = df_all_player_fixtures['opponent_team_id'].map(df_teams['name'])

In [11]:
df_all_player_fixtures.tail(20)

,id,code,team_h,team_h_score,team_a,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,player_id,name,team_id,opponent_team_id,team,opponent_team
1040,229,2270255,4,None,6,None,29,False,0,False,2022-10-30T14:00:00Z,Omgång 29,False,521,Momodou Sonko,6,4,BK Häcken,IFK Göteborg
1041,234,2270261,6,None,9,None,30,False,0,False,2022-11-06T14:00:00Z,Omgång 30,True,521,Momodou Sonko,6,9,BK Häcken,IFK Norrköping
1042,226,2270258,17,None,3,None,29,False,0,False,2022-10-30T14:00:00Z,Omgång 29,False,522,Hannes Sveijer,3,17,IK Sirius,GIF Sundsvall
1043,238,2270265,3,None,13,None,30,False,0,False,2022-11-06T14:00:00Z,Omgång 30,True,522,Hannes Sveijer,3,13,IK Sirius,Varbergs BoIS
1044,230,2270256,9,None,7,None,29,False,0,False,2022-10-31T18:10:00Z,Omgång 29,True,523,Moutaz Neffati,9,7,IFK Norrköping,Djurgården
1045,234,2270261,6,None,9,None,30,False,0,False,2022-11-06T14:00:00Z,Omgång 30,False,523,Moutaz Neffati,9,6,IFK Norrköping,BK Häcken
1046,226,2270258,17,None,3,None,29,False,0,False,2022-10-30T14:00:00Z,Omgång 29,True,524,Edwin Dellkrans,17,3,GIF Sundsvall,IK Sirius
1047,239,2270266,10,None,17,None,30,False,0,False,2022-11-06T14:00:00Z,Omgång 30,False,524,Edwin Dellkrans,17,10,GIF Sundsvall,Kalmar FF
1048,228,2270254,5,None,19,None,29,False,0,False,2022-10-31T18:00:00Z,Omgång 29,False,525,Simon Bengtsson,19,5,Helsingborgs IF,IF Elfsborg
1049,236,2270263,19,None,2,None,30,False,0,False,2022-11-06T14:00:00Z,Omgång 30,True,525,Simon Bengtsson,19,2,Helsingborgs IF,Hammarby


In [12]:
df_all_player_fixtures.to_csv('data/during-season/player_fixtures.csv', index=False)